<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/mapper-soilCondition"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://drive.google.com/drive/folders/1mEQBfa-tVViVWFt27XzUP4Wr19u1iuZm"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping soil condition | Extract predictor variables using Google Earth Engine  

**Author:** Jenny Hanssen, Willeke A'Campo, Zander Venter

**Description:** Script to get terrain variables (min, max, and mean elevation, slope, and aspect) over each wetland and store them as properties before exporting to csv.

## Connect to Earth Engine 

In [ ]:
# The earth engine api (ee) is standard in Google Colab
import ee
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
try:
    ee.Initialize() # Try to initialize earth engine
    print("Earth Engine is authenticated.")
except ee.EEException:
    ee.Authenticate() # Authenticate earth engine if initialization fails
    ee.Initialize() # init again 

## Import Data

In [ ]:
# Import Admin boundaries 
no_country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Norway'))
no_province = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Norway'))

# Import mires fc
table_mires = ee.FeatureCollection("projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var")

# Import the digital terrain model 'dtm10'
dtm10 = ee.Image("users/rangelandee/NINA/Raster/Fenoscandia_DTM_10m").rename('elevation')

## Display Data on Map

In [ ]:
# Create Map object
Map = geemap.Map(center=[60, 8], zoom=5)

# Center the map using the norway polygon
Map.center_object(no_country, 5)

### Styling

In [ ]:
# style provinces
style_pr = {'color': '444444', 
            'width': 0.5, 
            'lineType': 'solid', 
            'fillColor': '00000000'
            }

# style wetland polygons
style_my ={
    'color': '#FF007F', 
    'width': 0.8, 
    'lineType': 'solid', 
    'fillColor': '00000000', 
    }

# Set visualization parameters as a dictionary
vis_params_dem = {
    'min': 0, 
    'max': 4000, 
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
    }

### Add layers to Map 

In [ ]:
# BASEMAP
#Map.add_basemap('Esri.WorldGrayCanvas')
Map.add_basemap('Esri.WorldShadedRelief')

# DEM 
Map.addLayer(dtm10, vis_params_dem, 'DTM 10m', True, 0.3)

# WETLAND POLYGONS
Map.addLayer(table_mires.style(**style_my), {}, 'Mires')

# PROVINCEBORDER
Map.addLayer(no_country.style(**style_pr), {}, "Norway Fylker")

# Center the map using the norway polygon
Map.center_object(no_country, 6)
Map

## Data Analysis
### Extract terrain variables 

In [ ]:
# Function to extract terrain variables
def extractTerrainVariables(feature):
  minElevation = dtm10.reduceRegion(ee.Reducer.min(), feature.geometry()).get('elevation')
  maxElevation = dtm10.reduceRegion(ee.Reducer.max(), feature.geometry()).get('elevation')
  avgElevation = dtm10.reduceRegion(ee.Reducer.mean(), feature.geometry()).get('elevation')

  slopeAndAspect = ee.Terrain.products(dtm10).select(['slope', 'aspect'])
  avgSlope = slopeAndAspect.select('slope').reduceRegion(ee.Reducer.mean(), feature.geometry()).get('slope')
  avgAspect = slopeAndAspect.select('aspect').reduceRegion(ee.Reducer.mean(), feature.geometry()).get('aspect')

  return feature.set({
    'minElevation': minElevation,
    'maxElevation': maxElevation,
    'avgElevation': avgElevation,
    'avgSlope': avgSlope,
    'avgAspect': avgAspect
  })


# Map the function over the feature collection
miresWithTerrainVariables = table_mires.map(extractTerrainVariables)

Display the output data 

In [ ]:
import pandas as pd
df = pd.DataFrame(miresWithTerrainVariables.limit(10).getInfo()['features'])

In [ ]:
props_df = pd.json_normalize(df['properties'])
new_df = pd.concat([df,props_df], axis=1)
new_df.drop('properties',axis=1,inplace=True)
new_df.drop('geometry',axis=1,inplace=True)
display(new_df)

## Export the output data 

In [ ]:
# Store export options in dictionary
output_folder = 'GEE_output'
export_options = {
  'collection': miresWithTerrainVariables,
  'description': 'mires_terrain',
  'fileFormat': 'CSV',
  'folder': output_folder
  }

In [ ]:
import time
# Export table to Google Drive
task = ee.batch.Export.table.toDrive(**export_options)
task.start()

# Get export task status
status = task.status()['state']
while status == 'READY' or status == 'RUNNING':
    time.sleep(5)
    status = task.status()['state']

# Print export location
if status == 'COMPLETED':
    print(f"Export completed. File is stored in ...Google Drive/{output_folder}")
else:
    print("Export failed.")


### Clear Memory 

In [ ]:
# Clear data from memory
del miresWithTerrainVariables